In [ ]:
import sys
import os
import importlib.util
import argparse
import optuna
import importlib

from zfa.model_training.optic_flow_train import train

In [ ]:
class Args:
    def __init__(self):
        self.name = 'optic_flow_model_test'
        self.save_checkpoint = False
        self.num_epochs = 1
        self.size = 16 # Change this 
        self.gpus = [0]
        self.mixed_precision = False
        self.epsilon = 1e-8
        self.clip = 1.0
        self.add_noise = True        
        self.checkpoint_path = None 
        self.test_batch_size = 1
        self.batch_size = 1

args = Args()


In [ ]:
# Input size for resizing
    size = args.size  

    # Define augmentation parameters for training
    aug_params_train = aug_params = {
        
        'crop_size': (256, 448), 
        'min_scale': -0.2,        # Scale down to 80% of the original size
        'max_scale': 0.2,         # Scale up to 120% of the original size
        'do_flip': True, 
        'size': size          
    }
    # Initialize datasets based on the split
    print("Fetching datasets optic_flow_fetch_dataloader")
    train_dataset = OpticFlowDataset(split='train',aug_params=aug_params_train)

In [ ]:
# Train model with given hyperparameters
one, two, three = train(args)
print(f'EPE: {one}, F1: {two}, F2: {three}')

Training model
Initializing model
Loading checkpoint
Fetching data loaders
Fetching datasets optic_flow_fetch_dataloader
Init OpticFlowDataset
Augmenting   OpticFlowDataset
Matching files  OpticFlowDataset


Matching files:   0%|          | 0/1184 [00:00<?, ?it/s]

Matching files:  83%|████████▎ | 980/1184 [10:53<02:16,  1.50it/s]  

KeyboardInterrupt



In [ ]:
# Call the train function
# Objective function for Optuna
def objective(trial):
    args = Args()

    # Use Optuna to suggest the value for 'size'
    'lr': trial.suggest_categorical('lr', [1e-5, 1e-4, 1e-3, 1e-2, 1e-1]),
    'batch_size': trial.suggest_categorical('batch_size', [64, 128, 256]),
    'wdecay': trial.suggest_categorical('wdecay', [1e-4, 1e-3, 1e-2])


    # Call the train function from the script
    path, train_losses, val_losses = module.train(args) 

    # Return the objective value to minimize or maximize
    return val_losses[-1]

In [4]:
# Create the Optuna study
study = optuna.create_study(direction='minimize', study_name="Optic_Flow_ntrials45_size16") 

# Run the optimization
study.optimize(objective, n_trials=45)  

# Print the best result
print(f"Best trial: {study.best_trial.value}")
print(f"Best parameters: {study.best_trial.params}")

[I 2024-10-09 21:55:34,588] A new study created in memory with name: Optic_Flow_ntrials5_numsteps10000 


Parameter Count: 1277817
[   100,  0.0000046] 39213935.2800, 
[   200,  0.0000084] 39586503.6000, 
[   300,  0.0000122] 39262894.0400, 
[   400,  0.0000160] 38869370.4800, 
[   500,  0.0000198] 38599647.5600, 
[   600,  0.0000198] 38406608.3200, 
[   700,  0.0000196] 38275201.2800, 
[   800,  0.0000194] 38034724.4200, 
[   900,  0.0000192] 37602086.0000, 
[  1000,  0.0000190] 37314164.2600, 
[  1100,  0.0000188] 37167239.0600, 
[  1200,  0.0000186] 37332625.6200, 
[  1300,  0.0000183] 36949593.6600, 
[  1400,  0.0000181] 37113912.7000, 
[  1500,  0.0000179] 37067720.7000, 
[  1600,  0.0000177] 36298387.4400, 
[  1700,  0.0000175] 36658991.5600, 
[  1800,  0.0000173] 36188631.8400, 
[  1900,  0.0000171] 36622202.1800, 
[  2000,  0.0000169] 36568831.2000, 
[  2100,  0.0000167] 35904656.3800, 
[  2200,  0.0000165] 36273585.1200, 
[  2300,  0.0000163] 35799002.6800, 
[  2400,  0.0000161] 35973277.1400, 
[  2500,  0.0000158] 35916659.3000, 
[  2600,  0.0000156] 35540314.5600, 
[  2700,  0.0

[W 2024-10-09 23:20:49,730] Trial 0 failed with parameters: {'size': 64} because of the following error: RuntimeError('The size of tensor a (64) must match the size of tensor b (85) at non-singleton dimension 3').
Traceback (most recent call last):
  File "/home/akirscht/miniconda3/envs/zfa/lib/python3.12/site-packages/optuna/study/_optimize.py", line 197, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "/tmp/ipykernel_3633320/4011041439.py", line 12, in objective
    epe = module.train(args)  # Assuming your train function returns accuracy or a metric to optimize
          ^^^^^^^^^^^^^^^^^^
  File "/home/akirscht/zebrafish_agent/scripts/python_scripts/akt-train-optic-flow.py", line 211, in train
    results = validate_model(model, dataloaders['val'], device=device)
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/akirscht/miniconda3/envs/zfa/lib/python3.12/site-packages/torch/utils/_contextlib.py", line 116,

RuntimeError: The size of tensor a (64) must match the size of tensor b (85) at non-singleton dimension 3